In [1]:
!pip install langchain
!pip install PyPDF2
!pip install pypdf
!pip install tiktoken
!pip install openai
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import QAGenerationChain
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
import os
import re
import json
import time
from PyPDF2 import PdfReader
import csv

In [3]:
from google.colab import files

uploaded = files.upload()

Saving Big Mac Index.pdf to Big Mac Index.pdf


In [9]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [5]:
def generate_true_false_questions(text):
    """
    Generate true/false questions based on the provided text.

    Args:
        text (str): The text content to base the questions on.

    Returns:
        list: A list of generated true/false questions.
    """
    # Initialize a language model for question generation
    llm_ques_gen = ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo")

    # Define the prompt template for true/false questions
    true_false_prompt_template = """
    You are an expert at creating true/false questions based on the provided text.
    Your goal is to test the knowledge of coders or programmers on the content below:

    ------------
    {text}
    ------------

    Create true/false questions that will assess understanding.
    Ensure questions are clear and concise.

    QUESTIONS:
    """

    # Create a prompt template with the specified input variable
    prompt = PromptTemplate(input_variables=["text"], template=true_false_prompt_template)

    # Initialize an LLMChain for question generation
    llmChain = LLMChain(llm=ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo"), prompt=prompt)

    # Generate questions using the language model
    questions = llmChain.run(text)

    # Parse and format the generated questions
    ques = []
    questions = re.findall(r'(\d+)\.\s+(.*)', questions)
    for number, question in questions:
        ques.append(f"{number}. {question}")

    return ques

def generate_multiple_choice_questions(text):
    """
    Generate multiple-choice questions based on the provided text.

    Args:
        text (str): The text content to base the questions on.

    Returns:
        list: A list of generated multiple-choice questions.
    """
    # Define the prompt template for multiple-choice questions
    prompt_template = """
    You are preparing multiple-choice questions based on the following text chunk:
    ------------
    {text}
    ------------

    Generate multiple-choice questions that cover important concepts.
    Provide clear and relevant options.

    QUESTIONS:
    """

    # Create a prompt template with the specified input variable
    prompt = PromptTemplate(input_variables=["text"], template=prompt_template)

    # Initialize an LLMChain for question generation
    llmChain = LLMChain(llm=ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo"), prompt=prompt)

    # Generate questions using the language model
    questions = llmChain.run(text)

    # Split generated questions into blocks and format them
    question_blocks = questions.strip().split('\n\n')
    question_array = []
    for block in question_blocks:
        lines = block.strip().split('\n')
        question_number, question_content = lines[0].split('. ', 1)
        options = ' '.join(f"{line.strip()}" for line in lines[1:])
        formatted_question = f"{question_number}. {question_content} {options}"
        question_array.append(formatted_question)

    return question_array

def generate_one_word_answer_questions(text):
    """
    Generate one-word answer questions based on the provided text.

    Args:
        text (str): The text content to base the questions on.

    Returns:
        list: A list of generated one-word answer questions.
    """
    # Define the prompt template for one-word answer questions
    prompt_template = """
    You are preparing one-word answer questions based on the following text chunk:
    ------------
    {text}
    ------------

    Generate one-word answer questions that target key information.
    Keep questions precise and focused.

    QUESTIONS:
    """

    # Create a prompt template with the specified input variable
    prompt = PromptTemplate(input_variables=["text"], template=prompt_template)

    # Initialize an LLMChain for question generation
    llmChain = LLMChain(llm=ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo"), prompt=prompt)

    # Generate questions using the language model
    questions = llmChain.run(text)

    # Split generated questions into a list
    questions = questions.split('\n')

    return questions


In [10]:
def process_text_and_generate_questions(text):
    splitter = TokenTextSplitter(model_name='gpt-3.5-turbo', chunk_size=10000, chunk_overlap=200)
    text_chunks = splitter.split_text(text)

    all_questions = {
        "true_false": [],
        "multiple_choice": [],
        "one_word_answer": []
    }

    for chunk in text_chunks:
        if chunk.strip():
            # Generate questions for each chunk and append to respective lists
            all_questions["true_false"] = generate_true_false_questions(chunk)
            all_questions["multiple_choice"] = generate_multiple_choice_questions(chunk)
            all_questions["one_word_answer"] = generate_one_word_answer_questions(chunk)

    return all_questions

# Assuming correct implementation of PyPDFLoader and loader.load() to extract text
loader = PyPDFLoader('/content/Big Mac Index.pdf')
data = loader.load()

question_gen = []

for page in data:
    question_gen.append(page.page_content)  # Accumulate page content in a list

# Join all page contents into a single string
generated_text = '\n'.join(question_gen)

# Process the concatenated text to generate questions
generated_questions = process_text_and_generate_questions(generated_text)

print("True/False Questions:")
print(generated_questions["true_false"])

print("\nMultiple-Choice Questions:")
print(generated_questions["multiple_choice"])

print("\nOne-Word Answer Questions:")
print(generated_questions["one_word_answer"])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


True/False Questions:
['1. The Big Mac Index was introduced in The Economist in September 1986 by Pam Woodall as a serious tool for exchange rate evaluation. (True/False)', '2. The Big Mac Index compares the relative price worldwide to purchase a Quarter Pounder with Cheese. (True/False)', '3. The Big Mac Index is based on the concept of Purchasing Power Parity (PPP). (True/False)', "4. The Big Mac Index provides a method to analyze a currency's level of under/over-valuation against a base currency. (True/False)", "5. The Big Mac Index is limited by geographical coverage due to the presence of the McDonald's franchise. (True/False)"]

Multiple-Choice Questions:
["1. What is the purpose of The Big Mac Index? A. To measure the popularity of McDonald's Big Mac sandwich B. To calculate an implied exchange rate between two currencies C. To determine the nutritional value of the Big Mac D. To analyze the cost of advertising for McDonald's", '2. How is the implied exchange rate calculated in 

In [14]:
def file_processing(file_path):

    # Load data from PDF
    loader = PyPDFLoader(file_path)
    data = loader.load()

    question_gen = ''

    for page in data:
        question_gen += page.page_content

    splitter_ques_gen = TokenTextSplitter(
        model_name = 'gpt-3.5-turbo',
        chunk_size = 10000,
        chunk_overlap = 200
    )

    chunks_ques_gen = splitter_ques_gen.split_text(question_gen)

    document_ques_gen = [Document(page_content=t) for t in chunks_ques_gen]

    splitter_ans_gen = TokenTextSplitter(
        model_name = 'gpt-3.5-turbo',
        chunk_size = 1000,
        chunk_overlap = 100
    )


    document_answer_gen = splitter_ans_gen.split_documents(
        document_ques_gen
    )

    return document_ques_gen, document_answer_gen

In [36]:
def generate_true_false_answers(text, answer_generation_chain):
    """
    Generate a true/false answer based on the provided text using a language model.

    Args:
        text (str): The text containing the statement to answer.
        llm_model: The language model used for generating answers.

    Returns:
        str: The generated true/false answer (either 'true' or 'false').
    """


    result = answer_generation_chain({"query": text + 'Only Answer.'})

    # Clean up the generated answer (convert to lowercase and strip whitespace)
    cleaned_answer = result['result'].strip().lower()

    return cleaned_answer

def generate_multiple_choice_answers(text, answer_generation_chain):
    """
    Generate a multiple-choice answer based on the provided text using a language model.

    Args:
        text (str): The text containing the multiple-choice question.
        llm_model: The language model used for generating answers.

    Returns:
        str: The generated multiple-choice answer (e.g., 'A', 'B', 'C', etc.).
    """

    result = answer_generation_chain({"query": text + 'Please select the correct option (A, B, C, etc.) for the following question. Only Answer.'})

    cleaned_answer = result['result'].strip().upper()

    return cleaned_answer

def generate_one_word_answers(text, answer_generation_chain):
    """
    Generate a one-word answer based on the provided text using a language model.

    Args:
        text (str): The text containing the question requiring a one-word answer.
        llm_model: The language model used for generating answers.

    Returns:
        str: The generated one-word answer.
    """

    result = answer_generation_chain({"query": text + 'Please provide a one-word answer to the following question. Only Answer.'})


    # Clean up the generated answer (convert to uppercase and strip whitespace)
    cleaned_answer = result['result'].strip().lower()

    return cleaned_answer


In [39]:
def generate_ans_for_generated_ques(generated_questions):
    """
    Generate answers for the generated questions and write them to a CSV file.

    Args:
        generated_questions (dict): A dictionary containing lists of generated questions.

    Returns:
        None
    """

    document_ques_gen, document_answer_gen = file_processing('/content/Big Mac Index.pdf')

    embeddings = OpenAIEmbeddings()

    vector_store = FAISS.from_documents(document_answer_gen, embeddings)

    llm_answer_gen = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")

    answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen,
                                                chain_type="stuff",
                                                retriever=vector_store.as_retriever())


    # Dictionary to store question-answer mappings
    question_answer_mapping = {}

    # Generate answers for true/false questions
    # Add section header for true/false questions and answers
    question_answer_mapping["=== TRUE/FALSE QUESTIONS ==="] = "=== ANSWERS BELOW ==="
    for question in generated_questions["true_false"]:
        generated_answer = generate_true_false_answers(question, answer_generation_chain)
        question_answer_mapping[question] = generated_answer
    # print(question_answer_mapping)


    # # Add section header for multiple-choice questions and answers
    question_answer_mapping["=== MULTIPLE CHOICE QUESTIONS ==="] = "=== ANSWERS BELOW ==="
    # Generate answers for multiple-choice questions
    for question in generated_questions["multiple_choice"]:
        generated_answer = generate_multiple_choice_answers(question, answer_generation_chain)
        question_answer_mapping[question] = generated_answer


    # Add section header for one-word answer questions and answers
    question_answer_mapping["=== ONE-WORD ANSWER QUESTIONS ==="] = "=== ANSWERS BELOW ==="
    # Generate answers for one-word answer questions
    for question in generated_questions["one_word_answer"]:
        generated_answer = generate_one_word_answers(question, answer_generation_chain)
        question_answer_mapping[question] = generated_answer

    print(question_answer_mapping)


    # # Write question-answer mappings to a CSV file
    with open('question_answer_mapping.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Question', 'Answer']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write CSV header
        writer.writeheader()

        # Write question-answer pairs to CSV rows
        for question, answer in question_answer_mapping.items():
            writer.writerow({'Question': question, 'Answer': answer})


In [40]:
generate_ans_for_generated_ques(generated_questions)

{'=== TRUE/FALSE QUESTIONS ===': '=== ANSWERS BELOW ===', '1. The Big Mac Index was introduced in The Economist in September 1986 by Pam Woodall as a serious tool for exchange rate evaluation. (True/False)': 'false', '2. The Big Mac Index compares the relative price worldwide to purchase a Quarter Pounder with Cheese. (True/False)': 'false', '3. The Big Mac Index is based on the concept of Purchasing Power Parity (PPP). (True/False)': 'true', "4. The Big Mac Index provides a method to analyze a currency's level of under/over-valuation against a base currency. (True/False)": 'true', "5. The Big Mac Index is limited by geographical coverage due to the presence of the McDonald's franchise. (True/False)": 'true', '=== MULTIPLE CHOICE QUESTIONS ===': '=== ANSWERS BELOW ===', "1. What is the purpose of The Big Mac Index? A. To measure the popularity of McDonald's Big Mac sandwich B. To calculate an implied exchange rate between two currencies C. To determine the nutritional value of the Big 